In [8]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.metrics import jaccard_score

In [9]:
from sklearn import preprocessing

file_path_train = r"D:\Download\zyFile\Cyberthreat_Cognitive_System\CTCS_Code\attack_datasets\NSL-KDD\KDDTrain+.txt"
file_path_test = r"D:\Download\zyFile\Cyberthreat_Cognitive_System\CTCS_Code\attack_datasets\NSL-KDD\KDDTest+.txt"
# 定义列名
data_columns = ["duration", "protocol_type", "service", "flag", "src_bytes",
                "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins",
                "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root",
                "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
                "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate",
                "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
                "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
                "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
                "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
                "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label", "difficulty"]
# 加载数据
traindata = pd.read_csv(file_path_train, header=None, names=data_columns)
testdata = pd.read_csv(file_path_test, header=None, names=data_columns)
attack_mapping = {}
with open(r'D:\Download\zyFile\Cyberthreat_Cognitive_System\CTCS_Code\attack_datasets\NSL-KDD\attack_name',
          'r') as file:
    for line in file:
        parts = line.strip().split(' ')
        if len(parts) == 2:
            attack, category = parts
            attack_mapping[attack] = category
# 然后像之前那样使用这个映射字典
traindata['label'] = traindata['label'].map(attack_mapping)
testdata['label'] = testdata['label'].map(attack_mapping)
# 找出只在训练集中出现的 service 类型
unique_service_train = set(traindata['service']) - set(testdata['service'])
# 删除训练集中存在但测试集中不存在的 service 类型的行
traindata = traindata[~traindata['service'].isin(unique_service_train)]
# 初始化LabelEncoder
le_protocol_type = preprocessing.LabelEncoder()
le_service = preprocessing.LabelEncoder()
le_flag = preprocessing.LabelEncoder()
le_labels = preprocessing.LabelEncoder()

# 删除train_data中的difficulty, label列
X_train = traindata.drop('label', axis=1)
X_train = X_train.drop('difficulty', axis=1)
# columns_to_drop = [data_columns[i] for i in [6, 8, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 27, 30]]
# X_train = X_train.drop(columns=columns_to_drop)
# 提取出训练集中的label标签
labels_train = traindata['label']

# 删掉测试集的标签项
X_test = testdata.drop('label', axis=1)
X_test = X_test.drop('difficulty', axis=1)
# X_test = X_test.drop(columns=columns_to_drop)
labels_test = testdata['label']

# 对训练集进行标签编码
X_train['protocol_type'] = le_protocol_type.fit_transform(X_train['protocol_type'])
X_train['service'] = le_service.fit_transform(X_train['service'])
X_train['flag'] = le_flag.fit_transform(X_train['flag'])
labels_train = le_labels.fit_transform(labels_train) + 1

# 使用相同的编码器对测试集进行标签编码
X_test['protocol_type'] = le_protocol_type.transform(X_test['protocol_type'])
X_test['service'] = le_service.transform(X_test['service'])
X_test['flag'] = le_flag.transform(X_test['flag'])
labels_test = le_labels.transform(labels_test) + 1

In [10]:
X = X_train
Y = labels_train
T = X_test
C = labels_test

In [11]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
scaler = Normalizer().fit(T)
testT = scaler.transform(T)

In [12]:
traindata = np.array(trainX)
trainlabel = np.array(Y)
testdata = np.array(testT)
testlabel = np.array(C)

In [13]:
model = LogisticRegression()
model.fit(traindata, trainlabel)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
print("***************************************************************")

***************************************************************


C:\Users\23076\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
# 假设 `expected` 和 `predicted` 分别是真实标签和模型预测标签
accuracy = accuracy_score(expected, predicted)
# 计算精确率 (precision)
precision = precision_score(expected, predicted)
# 计算召回率 (recall)
recall = recall_score(expected, predicted)
# 计算F1分数 (f1-score)
f1 = f1_score(expected, predicted)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [15]:
# fit a k-nearest neighbor model to the data
model = KNeighborsClassifier()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
# summarize the fit of the model

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")

KNeighborsClassifier()
[[5566 1184  579  307    0]
 [ 132 9425  146    6    2]
 [ 446  611 1364    0    0]
 [   4 2233  113  224    2]
 [  53   83   51    3   10]]
0.729
0.971
Accuracy
0.638
precision


NameError: name 'precision' is not defined

In [16]:
model = AdaBoostClassifier(n_estimators=100)
model.fit(traindata, trainlabel)

# make predictions
expected = testlabel
predicted = model.predict(testdata)
# summarize the fit of the model

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")

[[6099 1530    7    0    0]
 [ 565 9073   72    0    1]
 [1427  849  145    0    0]
 [ 255 2314    7    0    0]
 [  36  113   51    0    0]]
0.799
0.934
Accuracy
0.638
precision


NameError: name 'precision' is not defined